### Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import perceval as pcvl
import matplotlib.pyplot as plt
from datetime import datetime

from src.models import QCBM
from src.helpers import ParametrizedQuantumCircuit
from src.helpers.utils import gaussian_mixture_pdf, kl_divergence

### Configuration of the numerical experiment

Here we defined the initial state, whether we have photon number resolving (PNR) detectors, and the structure of the ansatz (number of variational blocks):

In [ ]:
# input config
input_state = pcvl.BasicState("|0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0>")
pnr = False
#arch = ["var", "var"]
arch = ["var"]

Here we define the number of trainings we will average on, the number of iterations per training, and the number of optimization steps. We also define the initial amount of samples (i.e. photons) injected into the QCBM, and the loss parameter $\eta$.

In [ ]:
# optimization config
run_count = 20
it_count = 100
opt_steps = 1000

sample_count = 200000
loss_parameter = 0.8

Now we define the first QCBM of this notebook: a lossy scenario where photon recycling is applied.

Further parameters about the ansatz, such as one_param_per_interferometer, can also be defined. We refer to the QCBM class for more details.

In [ ]:
pqc = ParametrizedQuantumCircuit(input_state.m, arch, same_params_in_var=False, one_param_per_interferometer=False)
bm_lossy = QCBM(parametrized_circuit=pqc, input_state=input_state, sample_count=sample_count, loss_parameter=loss_parameter, pnr=pnr, 
                use_samples_only = False, use_photon_recycling=True, miti=True, threshold_stats = True)

# get bin count of the distribution based on the output of the circuit
bin_count = bm_lossy.bin_count
print(bin_count)

We define the target distribution and the loss function:

In [ ]:
target_pdf_name = 'gaussian_mixture'
target_pdf = gaussian_mixture_pdf(bin_count)
target_space = np.arange(bin_count)

loss_name = 'kl'
loss_fun = kl_divergence()
#loss_fun = RBFMMD2(sigma_list=[0.25, 4.], basis = np.arange(bin_count))
#loss_fun = mmd_rbf(gamma = 0.25)
#loss_fun = tvd()

We assign this information to the QCBM.

In the case of using samples only in the loss function evaluation, use the commented lines below:

In [ ]:
bm_lossy.target_pdf = target_pdf
bm_lossy.loss_fun = loss_fun
bm_lossy.target_space = target_space

# if using samples only
#target_samples = sample_from_target_pdf(target_space, bm_lossy.sample_count, target_pdf)
#bm_lossy.target_samples = target_samples

Perform a couple of checks about the target probability distribution:

In [ ]:
plt.plot(target_pdf)

In [ ]:
sum(target_pdf)

### Lossy, with photon recycling

Now, let's start the training of this first QCBM (lossy, with photon recycling applied).

First, we have the option to use initialization parameters that were saved previously.

In [ ]:
# Load init parameters - if reusing saved parameters
path_init = ''
has_init_params_runs = False
if os.path.exists(path_init):
    has_init_params_runs = True
    init_df = pd.read_csv(path_init)
    init_params_runs = init_df.to_numpy()

In [ ]:
bm_lossy.get_loss()

In [ ]:
lossy_loss_runs = np.zeros((run_count, it_count))
lossy_tvd_runs = np.zeros((run_count, it_count))
lossy_mmd_runs = np.zeros((run_count, it_count))
lossy_js_runs = np.zeros((run_count, it_count))
lossy_params_runs = np.zeros((run_count, len(bm_lossy.pqc.var_param_map)))

# If we wish to save the init parameters
if not has_init_params_runs:
    init_params_runs = []

# Set counter
i = 0

while i < run_count:
    if has_init_params_runs:
        init_params_lossy = init_params_runs[i]
        bm_lossy.pqc.init_params(red_factor = 1, init_var_params = init_params_lossy)
    else:
        init_params_lossy = bm_lossy.pqc.init_params()
    
    print('Initialization OK')
    
    try:
        loss_progress, params_progress, metric_tvd, metric_mmd, metric_js = bm_lossy.fit(opt_steps, it_count, silent=True)
        lossy_loss_runs[i, :] = loss_progress
        lossy_params_runs[i, :] = params_progress[-1]
        
        # Extra metrics saved for evaluation of the models:
        lossy_tvd_runs[i, :] = metric_tvd
        lossy_mmd_runs[i, :] = metric_mmd
        lossy_js_runs[i, :] = metric_js
        
        print('Training instance OK')
        i += 1
        
        # If we wish to save the init parameters
        init_params_runs.append(init_params_lossy)
        
    except:
        continue

### Lossy, without photon recycling

Then, we train a lossy QCBM without photon recycling.

We define it, and assign to it the same target distribution and same loss function as the first QCBM.

In [ ]:
bm_lossy_noPR = QCBM(parametrized_circuit=pqc, input_state=input_state, sample_count=sample_count, loss_parameter=loss_parameter, pnr=pnr, 
                     use_samples_only = False, use_photon_recycling=True, miti=False, threshold_stats = True)

bm_lossy_noPR.target_pdf = target_pdf
bm_lossy_noPR.loss_fun = loss_fun
bm_lossy_noPR.target_space = target_space

# If using samples only:
#bm_lossless.target_samples = target_samples

In [ ]:
bm_lossy_noPR.get_loss()

In [ ]:
lossynoPR_loss_runs = np.zeros((run_count, it_count))
lossynoPR_tvd_runs = np.zeros((run_count, it_count))
lossynoPR_mmd_runs = np.zeros((run_count, it_count))
lossynoPR_js_runs = np.zeros((run_count, it_count))
lossynoPR_params_runs = np.zeros((run_count, len(bm_lossy_noPR.pqc.var_param_map)))

# Set counter
i = 0

while i < run_count:
    # If using saved init parameters:
    init_params_lossy = init_params_runs[i]
    bm_lossy_noPR.pqc.init_params(red_factor = 1, init_var_params = init_params_lossy)
    
    # If not:
    #bm_lossless.pqc.init_params()
    
    print('Initialization OK')
    
    try:
        loss_progress, params_progress, metric_tvd, metric_mmd, metric_js = bm_lossy_noPR.fit(opt_steps, it_count, silent=True)
        lossynoPR_loss_runs[i, :] = loss_progress
        lossynoPR_params_runs[i, :] = params_progress[-1]
        
        # Extra metrics saved for evaluation of the models:
        lossynoPR_tvd_runs[i, :] = metric_tvd
        lossynoPR_mmd_runs[i, :] = metric_mmd
        lossynoPR_js_runs[i, :] = metric_js
        
        print('Training instance OK')
        i += 1
        
    except:
        continue

### Lossless case

Then, we train a QCBM without losses. Photon recycling is thus not necessary.

We define it, and assign to it the same target distribution and same loss function as the first and second QCBMs.

In [ ]:
bm_lossless = QCBM(parametrized_circuit=pqc, input_state=input_state, sample_count=sample_count, loss_parameter=0.0, pnr=pnr, 
                     use_samples_only = False, use_photon_recycling=False)

bm_lossless.target_pdf = target_pdf
bm_lossless.loss_fun = loss_fun
bm_lossless.target_space = target_space

# If using samples only
#bm_lossless.target_samples = target_samples

In [ ]:
bm_lossless.get_loss()

In [ ]:
lossless_loss_runs = np.zeros((run_count, it_count))
lossless_tvd_runs = np.zeros((run_count, it_count))
lossless_mmd_runs = np.zeros((run_count, it_count))
lossless_js_runs = np.zeros((run_count, it_count))
lossless_params_runs = np.zeros((run_count, len(bm_lossless.pqc.var_param_map)))

# Set counter
i = 0

while i < run_count:
    # If using saved init parameters:
    init_params_lossy = init_params_runs[i]
    bm_lossless.pqc.init_params(red_factor = 1, init_var_params = init_params_lossy)
    
    # If not:
    #bm_lossless.pqc.init_params()
    
    print('Initialization OK')
    
    try:
        loss_progress, params_progress, metric_tvd, metric_mmd, metric_js = bm_lossless.fit(opt_steps, it_count, silent=True)
        lossless_loss_runs[i, :] = loss_progress
        lossless_params_runs[i, :] = params_progress[-1]
        
        # Extra metrics saved for evaluation of the models:
        lossless_tvd_runs[i, :] = metric_tvd
        lossless_mmd_runs[i, :] = metric_mmd
        lossless_js_runs[i, :] = metric_js
        
        print('Training instance OK')
        i += 1
        
    except:
        continue

### Analysis

Below we can load previously saved results, we can analyze results by looking at plots of the various metrics.

In [ ]:
# Load results
#path = ''
#df_lossy_loss = pd.read_csv(path + "lossy_loss.csv")
#df_lossynoPR_loss = pd.read_csv(path + "lossynoPR_loss.csv")
#df_lossless_loss = pd.read_csv(path + "lossless_loss.csv")

#lossy_loss_runs = df_lossy_loss.to_numpy()
#lossynoPR_loss_runs = df_lossynoPR_loss.to_numpy()
#lossless_loss_runs = df_lossless_loss.to_numpy()

Averaging over the trainings:

In [ ]:
# mean and std over all simulation runs
lossy_mean = lossy_loss_runs.mean(axis = 0)
lossy_std = lossy_loss_runs.std(axis = 0)

lossynoPR_mean = lossynoPR_loss_runs.mean(axis = 0)
lossynoPR_std = lossynoPR_loss_runs.std(axis = 0)

lossless_mean = lossless_loss_runs.mean(axis = 0)
lossless_std = lossless_loss_runs.std(axis = 0)

Checking one iteration in particular:

In [ ]:
#it_check = 0
#lossy_mean = lossy_loss_runs[it_check]
#lossynoPR_mean = lossynoPR_loss_runs[it_check]
#lossless_mean = lossless_loss_runs[it_check]

Plotting main loss function:

In [ ]:
# specifies a left cutoff of the plot to accentuate the differences between ideal and noisy runs 
iteration_cutoff = 0

x = np.arange(it_count)
plt.plot(x[iteration_cutoff:], lossy_mean[iteration_cutoff:], label = 'Lossy with photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossy_mean - lossy_std)[iteration_cutoff:], (lossy_mean + lossy_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossynoPR_mean[iteration_cutoff:], label = 'Lossy without photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossynoPR_mean - lossynoPR_std)[iteration_cutoff:], (lossynoPR_mean + lossynoPR_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossless_mean[iteration_cutoff:], label = 'No losses')
plt.fill_between(x[iteration_cutoff:], (lossless_mean - lossless_std)[iteration_cutoff:], (lossless_mean + lossless_std)[iteration_cutoff:], alpha=0.2)


plt.legend()

A few checks:

In [ ]:
# Number of parameters
len(bm_lossless.pqc.var_params)

In [ ]:
# Does the distribution sum to 1?
sum(bm_lossless.pdf())

In [ ]:
# Does the distribution sum to 1?
sum(bm_lossy.pdf())

In [ ]:
# Does the distribution sum to 1?
sum(bm_lossy_noPR.pdf())

Save results:

In [ ]:
# save results
path = 'results/results_' + datetime.now().strftime("%Y%m%d_%H%M%S") + '/'
os.mkdir(path)

experiment_parameters  = {"Ansatz": [str(arch)],
                          "Input": [str(input_state)],
                          "Nsamples": [str(bm_lossy.sample_count)],
                          "Bins": [str(bm_lossy.bin_count)],
                          "Loss": [loss_name],
                          "Target": [target_pdf_name],
                          "run_count": [str(run_count)],
                          "it_count": [str(it_count)],
                          "opt_steps": [str(opt_steps)],
                          "sample_count": [str(sample_count)],
                          "loss_parameter": [str(loss_parameter)],
                          "one_param_per_interferometer": [str(bm_lossy.pqc.one_param_per_interferometer)],
                          "same_params_in_var": [str(bm_lossy.pqc.same_params_in_var)],
                          "PNR": [str(pnr)]}

df_experiment_parameters = pd.DataFrame(data = experiment_parameters)
df_experiment_parameters.to_csv(path + "experiment_parameters.csv", index = False)

df_init = pd.DataFrame(data = init_params_runs)
df_init.to_csv(path + "init.csv", index = False)

In [ ]:
df_lossy_loss = pd.DataFrame(data = lossy_loss_runs)
df_lossy_loss.to_csv(path + "lossy_loss.csv", index = False)

df_lossy_params = pd.DataFrame(data = lossy_params_runs)
df_lossy_params.to_csv(path + "lossy_params.csv", index = False)

In [ ]:
df_lossynoPR_loss = pd.DataFrame(data = lossynoPR_loss_runs)
df_lossynoPR_loss.to_csv(path + "lossynoPR_loss.csv", index = False)

df_lossynoPR_params = pd.DataFrame(data = lossynoPR_params_runs)
df_lossynoPR_params.to_csv(path + "lossynoPR_params.csv", index = False)

In [ ]:
df_lossless_loss = pd.DataFrame(data = lossless_loss_runs)
df_lossless_loss.to_csv(path + "lossless_loss.csv", index = False)

df_lossless_params = pd.DataFrame(data = lossless_params_runs)
df_lossless_params.to_csv(path + "lossless_params.csv", index = False)

### Check other metrics

Here we look at some extra metrics that we saved during the training.

In [ ]:
# mean and std over all simulation runs

# TVD 
lossy_tvd_mean = lossy_tvd_runs.mean(axis = 0)
lossy_tvd_std = lossy_tvd_runs.std(axis = 0)

lossynoPR_tvd_mean = lossynoPR_tvd_runs.mean(axis = 0)
lossynoPR_tvd_std = lossynoPR_tvd_runs.std(axis = 0)

lossless_tvd_mean = lossless_tvd_runs.mean(axis = 0)
lossless_tvd_std = lossless_tvd_runs.std(axis = 0)

# MMD 
lossy_mmd_mean = lossy_mmd_runs.mean(axis = 0)
lossy_mmd_std = lossy_mmd_runs.std(axis = 0)

lossynoPR_mmd_mean = lossynoPR_mmd_runs.mean(axis = 0)
lossynoPR_mmd_std = lossynoPR_mmd_runs.std(axis = 0)

lossless_mmd_mean = lossless_mmd_runs.mean(axis = 0)
lossless_mmd_std = lossless_mmd_runs.std(axis = 0)

# JS distance
lossy_js_mean = lossy_js_runs.mean(axis = 0)
lossy_js_std = lossy_js_runs.std(axis = 0)

lossynoPR_js_mean = lossynoPR_js_runs.mean(axis = 0)
lossynoPR_js_std = lossynoPR_js_runs.std(axis = 0)

lossless_js_mean = lossless_js_runs.mean(axis = 0)
lossless_js_std = lossless_js_runs.std(axis = 0)

Checking one iteration in particular:

In [ ]:
#it_check = 0
#lossy_tvd_mean = lossy_tvd_runs[it_check]
#lossynoPR_tvd_mean = lossynoPR_tvd_runs[it_check]
#lossless_tvd_mean = lossless_tvd_runs[it_check]

Plot of the TVD:

In [ ]:
# specifies a left cutoff of the plot to accentuate the differences between ideal and noisy runs 
iteration_cutoff = 0

x = np.arange(it_count)
plt.plot(x[iteration_cutoff:], lossy_tvd_mean[iteration_cutoff:], label = 'Lossy with photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossy_tvd_mean - lossy_tvd_std)[iteration_cutoff:], (lossy_tvd_mean + lossy_tvd_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossynoPR_tvd_mean[iteration_cutoff:], label = 'Lossy without photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossynoPR_tvd_mean - lossynoPR_tvd_std)[iteration_cutoff:], (lossynoPR_tvd_mean + lossynoPR_tvd_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossless_tvd_mean[iteration_cutoff:], label = 'No losses')
plt.fill_between(x[iteration_cutoff:], (lossless_tvd_mean - lossless_tvd_std)[iteration_cutoff:], (lossless_tvd_mean + lossless_tvd_std)[iteration_cutoff:], alpha=0.2)


plt.legend()

Checking one iteration in particular:

In [ ]:
#it_check = 0
#lossy_mmd_mean = lossy_mmd_runs[it_check]
#lossynoPR_mmd_mean = lossynoPR_mmd_runs[it_check]
#lossless_mmd_mean = lossless_mmd_runs[it_check]

Plot of the MMD:

In [ ]:
# specifies a left cutoff of the plot to accentuate the differences between ideal and noisy runs 
iteration_cutoff = 0

x = np.arange(it_count)
plt.plot(x[iteration_cutoff:], lossy_mmd_mean[iteration_cutoff:], label = 'Lossy with photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossy_mmd_mean - lossy_mmd_std)[iteration_cutoff:], (lossy_mmd_mean + lossy_mmd_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossynoPR_mmd_mean[iteration_cutoff:], label = 'Lossy without photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossynoPR_mmd_mean - lossynoPR_mmd_std)[iteration_cutoff:], (lossynoPR_mmd_mean + lossynoPR_mmd_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossless_mmd_mean[iteration_cutoff:], label = 'No losses')
plt.fill_between(x[iteration_cutoff:], (lossless_mmd_mean - lossless_mmd_std)[iteration_cutoff:], (lossless_mmd_mean + lossless_mmd_std)[iteration_cutoff:], alpha=0.2)


plt.legend()

Checking one iteration in particular:

In [ ]:
#it_check = 0
#lossy_js_mean = lossy_js_runs[it_check]
#lossynoPR_js_mean = lossynoPR_js_runs[it_check]
#lossless_js_mean = lossless_js_runs[it_check]

Plot of the Jensen Shannon divergence:

In [ ]:
# specifies a left cutoff of the plot to accentuate the differences between ideal and noisy runs 
iteration_cutoff = 0

ax = plt.gca()
ax.set_ylim([0.0, 1.0])

x = np.arange(it_count)
plt.plot(x[iteration_cutoff:], lossy_js_mean[iteration_cutoff:], label = 'Lossy with photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossy_js_mean - lossy_js_std)[iteration_cutoff:], (lossy_js_mean + lossy_js_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossynoPR_js_mean[iteration_cutoff:], label = 'Lossy without photon recycling')
plt.fill_between(x[iteration_cutoff:], (lossynoPR_js_mean - lossynoPR_js_std)[iteration_cutoff:], (lossynoPR_js_mean + lossynoPR_js_std)[iteration_cutoff:], alpha=0.2)

plt.plot(x[iteration_cutoff:], lossless_js_mean[iteration_cutoff:], label = 'No losses')
plt.fill_between(x[iteration_cutoff:], (lossless_js_mean - lossless_js_std)[iteration_cutoff:], (lossless_js_mean + lossless_js_std)[iteration_cutoff:], alpha=0.2)


plt.legend(loc='upper right')

Save those metrics:

In [ ]:
df_lossy_tvd = pd.DataFrame(data = lossy_tvd_runs)
df_lossy_tvd.to_csv(path + "lossy_tvd.csv", index = False)

df_lossless_tvd = pd.DataFrame(data = lossless_tvd_runs)
df_lossless_tvd.to_csv(path + "lossless_tvd.csv", index = False)

df_lossynoPR_tvd = pd.DataFrame(data = lossynoPR_tvd_runs)
df_lossynoPR_tvd.to_csv(path + "lossynoPR_tvd.csv", index = False)

In [ ]:
df_lossy_mmd = pd.DataFrame(data = lossy_mmd_runs)
df_lossy_mmd.to_csv(path + "lossy_mmd.csv", index = False)

df_lossless_mmd = pd.DataFrame(data = lossless_mmd_runs)
df_lossless_mmd.to_csv(path + "lossless_mmd.csv", index = False)

df_lossynoPR_mmd = pd.DataFrame(data = lossynoPR_mmd_runs)
df_lossynoPR_mmd.to_csv(path + "lossynoPR_mmd.csv", index = False)

In [ ]:
df_lossy_js = pd.DataFrame(data = lossy_js_runs)
df_lossy_js.to_csv(path + "lossy_js.csv", index = False)

df_lossless_js = pd.DataFrame(data = lossless_js_runs)
df_lossless_js.to_csv(path + "lossless_js.csv", index = False)

df_lossynoPR_js = pd.DataFrame(data = lossynoPR_js_runs)
df_lossynoPR_js.to_csv(path + "lossynoPR_js.csv", index = False)